In [5]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import random as rnd
import seaborn as sns

# Cargamos la data
callcenter_df = pd.read_csv(r'C:\Users\flore\Data Science\02 Data FT20\Proyecto_Integrador_M05\1 Call Center\Call_Center_1999_DataSet.csv', sep=";")

# Ajustamos los dataTypes de cada columna
# Ajustando Datatypes
# callcenter_df['call_id'] = callcenter_df['call_id'].astype(int)
callcenter_df['vru.line'] = callcenter_df['vru.line'].astype(str)
callcenter_df['priority'] = callcenter_df['priority'].astype('category')
callcenter_df['outcome'] = callcenter_df['outcome'].astype('category')
callcenter_df['server'] = callcenter_df['server'].astype('category')

# customer_id
# Ajustamos el Customer_id column cambiando el datatype a integer & # Con esta transformacion, todavia resta corregir los NaN a cero, para identificar a los clientes regulares no identificados por el VRU
callcenter_df['customer_id'] = pd.to_numeric(callcenter_df['customer_id'], errors='coerce', downcast='integer') # con esto normalizamos la columna.

# ahora procedemos a identificar a los clientes prospectos con un id unico = 99999999999
callcenter_df.loc[(callcenter_df['type'] == 'NW'), 'customer_id'] = 999999999999
# Ahora quedaron como 0 todos los clientes, que son clientes y que no fueron identificados al momento de ingresar en el call center
callcenter_df['customer_id'] = pd.to_numeric(callcenter_df['customer_id'].fillna(0), downcast='integer')

# type
# drop this records where type == AA and it is not part of the call center process
indexes_to_drop = [538,13233,109637,242738,320222]
callcenter_df.drop(indexes_to_drop,inplace=True)
# Corregimos para normalizar todos 
index_list_TT = [68094,73097,95287,95360,95414,98245]
callcenter_df.loc[index_list_TT,'type'] = 'TT'

callcenter_df['type'] = callcenter_df['type'].astype('category')

# La columna 'date' esta como string, no tiene valores nulos ni erroneos, pero hay que primero convertirlo y luego cambiar el datatype
callcenter_df['date'] = pd.to_datetime(callcenter_df['date'])

# lo mismo vamos a hacer con todas las columnas que hacen referencia de tiempo.
callcenter_df['vru_entry'] =pd.to_datetime(callcenter_df['vru_entry'])
callcenter_df['vru_exit'] = pd.to_datetime(callcenter_df['vru_exit'])
callcenter_df['q_start'] = pd.to_datetime(callcenter_df['q_start'])
callcenter_df['q_exit'] = pd.to_datetime(callcenter_df['q_exit']) 
callcenter_df['ser_start'] = pd.to_datetime(callcenter_df['ser_start'])
callcenter_df['ser_exit'] = pd.to_datetime(callcenter_df['ser_exit'])

# eliminamos la columna startdate
callcenter_df = callcenter_df.drop('startdate',axis=1)

# Continuamos con el analisis estadistico y seguimos limpiando y normalizando
df = callcenter_df.copy()

# Se agrega una nueva columna para el mes
df['month'] = df['date'].dt.month
# Se agrega una nueva columna para el dia
df['day'] = df['date'].dt.day
# Extraemos el dia de la semana y creamos una columna nueva 'days_of_week'
df['days_of_week'] = df['date'].dt.day_name()

# customer_id
# Creamos una nueva columna con las tres categorias por cutomer_id performance
df['customerid_category'] = df['customer_id']
df.loc[(df['customerid_category'] == 0), 'customerid_category'] = 'CustNotId'
df.loc[(df['customerid_category'] == 999999999999), 'customerid_category'] = 'ProspectCustomer'
df.loc[(df['customerid_category'] != 'CustNotId') & (df['customerid_category'] != 'ProspectCustomer'), 'customerid_category'] = 'IdentCustomer'
df['customerid_category'] = df['customerid_category'].astype(str)

# corregimos los valores negativos en vru_time
df['vru_entry_corr'] = np.where(df['vru_entry'] > df['vru_exit'], df['vru_exit'], df['vru_entry'])
df['vru_exit_corr'] = np.where(df['vru_entry'] > df['vru_exit'], df['vru_entry'], df['vru_exit'])
df['vru_time_corr'] = (df['vru_exit_corr'] - df['vru_entry_corr']).dt.total_seconds()
df['vru_time_corr'] = df['vru_time_corr'].astype('int64')

# eliminamos los outliers de vru_time_corr
df = df[df['vru_time_corr'] <=1000]

# Agregamos una columna con los tiempos reales de espera 
def corregir_q_time(row):
    if row['priority'] == 2:
        modified_value = row['q_time'] - 90
        return max(modified_value, 0)
    else:
        return row['q_time']

# Agregamos una columna corregida al data set para el analisis
df['q_time_real'] = df.apply(corregir_q_time, axis=1)

# Eliminamos los outliers de q_time 
df = df[df['q_time'] < 20000]   #(3 registros)

df = df[df['q_time'] <= 1000] # 138 registros 

df = df[df['ser_time'] <= 1000] # 5790 registros

# Eliminación de server
df = df.drop('server', axis=1)

# Guardamos el DF en un .csv para poder trabajarlo en Power BI
output_path = r'C:\Users\flore\Data Science\02 Data FT20\Proyecto_Integrador_M05\1 Call Center\callcenter_ok.csv'
df.to_csv(output_path, index=False)

C:\Users\flore\AppData\Local\Temp\ipykernel_5092\3813584800.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  callcenter_df = pd.read_csv(r'C:\Users\flore\Data Science\02 Data FT20\Proyecto_Integrador_M05\1 Call Center\Call_Center_1999_DataSet.csv', sep=";")
c:\Users\flore\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\dtypes\cast.py:378: RuntimeWarning: invalid value encountered in cast
  new_result = trans(result).astype(dtype)
C:\Users\flore\AppData\Local\Temp\ipykernel_5092\3813584800.py:42: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  callcenter_df['vru_entry'] =pd.to_datetime(callcenter_df['vru_entry'])
C:\Users\flore\AppData\Local\Temp\ipykernel_5092\3813584800.py:43: UserWarning: Could not infer format, so each element will be parsed individually, falling back